In [1]:
import pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.utils import to_categorical
import feather
import random
import time

Using TensorFlow backend.


In [3]:
FILENAME = "Datasets/Round_2/preproccesed_outliers_1085575_col_11_threshold_09"
data = feather.read_dataframe("{}.feather".format(FILENAME))

## Run this if you want to use the start latitude and longitude

In [3]:
# Create one hot encoding labels
n = len(data)
labels = data['Severity']
labels_oh = np.zeros((n, 4))
for i in range(n):
    labels_oh[i, labels[i]-1] = 1
    
# Get the attributes
atts_oh = data.drop('Severity', axis=1)

In [4]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels_oh, test_size=0.33, random_state=30034)

In [7]:
EPOCHS = 20
BATCH_SIZE = 128

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.swish,input_shape=(X_train.shape[1],))) # Swish indeed performs better than sigmoid
model.add(tf.keras.layers.Dense(4, activation=tf.nn.softmax))

# Compile and print out summary of model
model.compile(loss='categorical_crossentropy', 
              optimizer='adam',
             metrics=['accuracy'])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                4410      
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 124       
Total params: 4,534
Trainable params: 4,534
Non-trainable params: 0
_________________________________________________________________


In [8]:
# Train the model

start = time.time()

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

end = time.time()
print(end - start)

Epoch 1/20
1431510/1431510 [==============================] - 29s 21us/sample - loss: 0.9818 - acc: 0.6927
Epoch 2/20
1431510/1431510 [==============================] - 30s 21us/sample - loss: 0.6602 - acc: 0.7098
Epoch 3/20
1431510/1431510 [==============================] - 30s 21us/sample - loss: 0.5656 - acc: 0.7119
Epoch 4/20
1431510/1431510 [==============================] - 30s 21us/sample - loss: 0.5618 - acc: 0.7127
Epoch 5/20
1431510/1431510 [==============================] - 31s 21us/sample - loss: 0.5602 - acc: 0.7130
Epoch 6/20
1431510/1431510 [==============================] - 31s 21us/sample - loss: 0.5587 - acc: 0.7142
Epoch 7/20
1431510/1431510 [==============================] - 31s 22us/sample - loss: 0.5579 - acc: 0.7146
Epoch 8/20
1431510/1431510 [==============================] - 31s 22us/sample - loss: 0.5572 - acc: 0.7152
Epoch 9/20
1431510/1431510 [==============================] - 33s 23us/sample - loss: 0.5569 - acc: 0.7152
Epoch 10/20
1431510/1431510 [========

In [9]:
# Testing the model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\ntest_acc: ', test_acc)

705073/705073 [==============================] - 18s 26us/sample - loss: 0.5521 - acc: 0.7192

test_acc:  0.71916384


## Remove start lat and start long

In [4]:
new_data = data.drop(['Start_Lat', 'Start_Lng'], axis=1)

# Create one hot encoding labels
n = len(new_data)
labels = new_data['Severity']
labels_oh = np.zeros((n, 4))
for i in range(n):
    labels_oh[i, labels[i]-1] = 1
    
# Get the attributes
atts_oh = new_data.drop('Severity', axis=1)

In [5]:
# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(atts_oh, labels_oh, test_size=0.33, random_state=30034)

In [8]:
start = time.time()

model.fit(X_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

end = time.time()
print(end - start)

Instructions for updating:
Use tf.cast instead.
Epoch 1/20
1431510/1431510 [==============================] - 33s 23us/sample - loss: 0.5841 - acc: 0.7004
Epoch 2/20
1431510/1431510 [==============================] - 36s 25us/sample - loss: 0.5729 - acc: 0.7032
Epoch 3/20
1431510/1431510 [==============================] - 38s 26us/sample - loss: 0.5698 - acc: 0.7044
Epoch 4/20
1431510/1431510 [==============================] - 37s 26us/sample - loss: 0.5677 - acc: 0.7051
Epoch 5/20
1431510/1431510 [==============================] - 38s 26us/sample - loss: 0.5664 - acc: 0.7057
Epoch 6/20
1431510/1431510 [==============================] - 39s 27us/sample - loss: 0.5656 - acc: 0.7063
Epoch 7/20
1431510/1431510 [==============================] - 38s 27us/sample - loss: 0.5652 - acc: 0.7064
Epoch 8/20
1431510/1431510 [==============================] - 38s 26us/sample - loss: 0.5649 - acc: 0.7069
Epoch 9/20
1431510/1431510 [==============================] - 42s 29us/sample - loss: 0.5646 - a

In [9]:
# Testing the model 
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\ntest_acc: ', test_acc)

705073/705073 [==============================] - 19s 28us/sample - loss: 0.5671 - acc: 0.6919

test_acc:  0.69194824


## (Just analysing the count of each weather conditions)

In [23]:
count = 0
for col in data.columns:
    if col.startswith('Weather_Condition'):
        print(col[len('Weather_Condition_'):], sum(data[col]))
        count = count+1
print(count)

Blowing Dust 37
Blowing Dust / Windy 10
Blowing Sand 1
Blowing Snow 60
Blowing Snow / Windy 0
Clear 490250
Cloudy 142594
Cloudy / Windy 1421
Drifting Snow 0
Drizzle 351
Drizzle / Windy 0
Drizzle and Fog 48
Dust Whirls 0
Fair 406728
Fair / Windy 2632
Fog 12842
Fog / Windy 9
Freezing Drizzle 0
Freezing Rain 0
Freezing Rain / Windy 0
Funnel Cloud 12
Hail 1
Haze 16806
Haze / Windy 9
Heavy Blowing Snow 0
Heavy Drizzle 23
Heavy Freezing Drizzle 0
Heavy Freezing Rain 0
Heavy Ice Pellets 0
Heavy Rain 375
Heavy Rain / Windy 3
Heavy Rain Shower 0
Heavy Rain Showers 0
Heavy Sleet 0
Heavy Smoke 0
Heavy Snow 125
Heavy Snow / Windy 3
Heavy Snow with Thunder 0
Heavy T-Storm 146
Heavy T-Storm / Windy 0
Heavy Thunderstorms and Rain 45
Heavy Thunderstorms and Snow 0
Heavy Thunderstorms with Small Hail 0
Ice Pellets 19
Light Blowing Snow 0
Light Drizzle 5170
Light Drizzle / Windy 6
Light Fog 2
Light Freezing Drizzle 360
Light Freezing Fog 202
Light Freezing Rain 378
Light Freezing Rain / Windy 0
Light Ha